In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, col, regexp_replace, split, lower

try:
    spark = SparkSession.builder \
        .appName("Proyecto SparkSQL") \
        .config("spark.jars.packages", "org.postgresql:postgresql:42.7.4") \
        .config("spark.driver.extraClassPath", "/path/to/postgresql-42.7.4.jar") \
        .config("spark.executor.extraClassPath", "/path/to/postgresql-42.7.4.jar") \
        .getOrCreate()
    print("SparkSession creada:", spark.version)
except Exception as e:
    print("Error al crear SparkSession:", str(e))


# Load CSV file OIJ.CSV
df_oij = spark.read.csv("C:\\CODE\\SPARK\\spark\\data\\OIJ.csv", header=True, inferSchema=True)

# Load CSV file INEC.CSV
df_inec = spark.read.csv("C:\\CODE\\SPARK\\spark\\data\\INEC.csv", header=True, inferSchema=True)

def district_lowercase(df, column_name):
    """
    Pasa los datos de una columna a minúsculas 
    
    Args:
        df: DataFrame de Spark
        column_name: Nombre de la columna a limpiar
        
    Returns:
        DataFrame con la columna con sus datos en minúscula
    """
    # usando la funcion lower, modifica los datos de la columna con la información en minúscula
    df_lower = df.withColumn(
        column_name,
        lower(column_name))
    
    
    return df_lower


# Uso:
# Para OIJ.csv:
#df_oij_clean = clean_district_column(df_oij, 'Distrito')

#lower Case

#df_oij_low = district_lowercase(df_oij_clean, 'Distrito')




# Show the first few rows of the DataFrame
#df_oij_low.show()

# Print the DataFrame schema
#df_oij_low.printSchema()

SparkSession creada: 3.5.3
+------+-----------+----------+-------------------+----------+--------------------+-------------+------------+-----------+----------+-------------+----+
|Delito|  SubDelito|     Fecha|            Victima|SubVictima|                Edad|         Sexo|Nacionalidad|  Provincia|    Canton|     Distrito|_c11|
+------+-----------+----------+-------------------+----------+--------------------+-------------+------------+-----------+----------+-------------+----+
|ASALTO|ARMA BLANCA|2011-01-10|18:00:00 - 20:59:59|   PERSONA|    PEATON [PERSONA]|Mayor de edad|      HOMBRE|  NICARAGUA|  SAN JOSE|   alajuelita|NULL|
|ASALTO|ARMA BLANCA|2011-02-02|03:00:00 - 05:59:59|   PERSONA|    PEATON [PERSONA]|Mayor de edad|      HOMBRE| COSTA RICA|   CARTAGO|     oreamuno|NULL|
|ASALTO|ARMA BLANCA|2011-10-23|21:00:00 - 23:59:59|   PERSONA|    PEATON [PERSONA]|Mayor de edad|      HOMBRE| COSTA RICA|   HEREDIA|      heredia|NULL|
|ASALTO|ARMA BLANCA|2011-05-19|18:00:00 - 20:59:59|   P